# Credit Card Fraud Detection with Machine Learning Models

## Ian Heung

In this project, I will use a dataset from Kaggle and develop machine learning models to determine whether a transaction is fraudulent or not. The dataset is highly unbalanced, with the number of fraud cases only accounting for **only 0.172%** of transactions. I will navigate the challenge of the class imbalance in the dataset and develop machine learning models for accurate classification of the transactions.

In [ ]:
#